In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import (
    col, lit, when, upper, lower, concat, substring, 
    sum, avg, count, max, round, row_number, rank, dense_rank, lag
)
from pyspark.sql.types import IntegerType

In [ ]:
def funcao_auxiliar():
    print("Java > Python")
    print("Professor Alisson tacomNADA")

var = 1
var = var + 20
funcao_auxiliar()

In [ ]:
spark = SparkSession.builder \
    .appName("primeiraAula") \
    .getOrCreate()

In [3]:
# Dados de teste
dados = [
    ("Expansão", "Patrik", 5800),
    ("Caroline", "Power BI", 9000),
    ("Ana Raquel", "Que eu?", 14500),
    ("Jaqueline", "Modelo", 1000),
    ("Jaqueline", "Spy", 24000),
    ("Andrei", "CEO", 0)
]
colunas = ["Nome", "Departamento", "Salario"]

df = spark.createDataFrame(dados, colunas)

In [4]:
df

DataFrame[Nome: string, Departamento: string, Salario: bigint]

In [5]:
df_filtrado = df.filter(col("Salario") > 14000)

In [6]:
df_filtrado

DataFrame[Nome: string, Departamento: string, Salario: bigint]

In [7]:
df_filtrado.show()

+----------+------------+-------+
|      Nome|Departamento|Salario|
+----------+------------+-------+
|Ana Raquel|     Que eu?|  14500|
| Jaqueline|         Spy|  24000|
+----------+------------+-------+



In [8]:
df_filtrado = df.filter(col("Salario") < 2000)

In [9]:
df_filtrado.show()

+---------+------------+-------+
|     Nome|Departamento|Salario|
+---------+------------+-------+
|Jaqueline|      Modelo|   1000|
|   Andrei|         CEO|      0|
+---------+------------+-------+



In [13]:
df.show()

+----------+------------+-------+
|      Nome|Departamento|Salario|
+----------+------------+-------+
|    Patrik|    Expansão|   5800|
|  Caroline|    Power BI|   9000|
|Ana Raquel|     Que eu?|  14500|
| Jaqueline|      Modelo|   1500|
|    Andrei|         CEO|      0|
+----------+------------+-------+



In [2]:
spark = SparkSession.builder \
    .appName("Final_Spark") \
    .master("local[*]") \
    .getOrCreate()


In [4]:

vendas_data = [
    (1, "2024-01-01", "Notebook Gamer", 5000.00, "Eletronicos", 101),
    (2, "2024-01-01", "Mouse Sem Fio", 50.00, "Acessorios", 102),
    (3, "2024-01-02", "Teclado Mecanico", 200.00, "Acessorios", 101),
    (4, "2024-01-02", "Monitor 24", 900.00, "Eletronicos", 103),
    (5, "2024-01-03", "Cadeira Gamer", 1200.00, "Moveis", 101),
    (6, "2024-01-03", "Mousepad", 30.00, "Acessorios", 104), # Cliente 104 não existe
    (7, "2024-01-04", "Notebook Office", 3000.00, "Eletronicos", 102)
]
df_vendas = spark.createDataFrame(vendas_data, ["id_venda", "data", "produto", "valor", "categoria", "id_cliente"])


clientes_data = [
    (101, "Julia Pigosso", "SP"),
    (102, "Alisson Felipe", "AC"),
    (103, "Chico Bento", "MG"),
    (105, "Ana Julia", "SP") # Cliente nunca comprou nada
]
df_clientes = spark.createDataFrame(clientes_data, ["id_cliente", "nome", "estado"])


In [5]:
# Criar coluna de desconto (10%)
# Calcular valor líquido (valor - desconto)
# Colocar categoria em maiúsculo
# Criar coluna de origem do dado (valor fixo "SISTEMA_A")

df_calculado = df_vendas \
    .withColumn("desconto", col("valor") * 0.10) \
    .withColumn("valor_liquido", col("valor") - col("desconto"))\
    .withColumn("categoria_upper", upper(col("categoria"))) \
    .withColumn("origem_dado", lit("SISTEMA_A"))

In [6]:
df_calculado.show()

+--------+----------+----------------+------+-----------+----------+--------+-------------+---------------+-----------+
|id_venda|      data|         produto| valor|  categoria|id_cliente|desconto|valor_liquido|categoria_upper|origem_dado|
+--------+----------+----------------+------+-----------+----------+--------+-------------+---------------+-----------+
|       1|2024-01-01|  Notebook Gamer|5000.0|Eletronicos|       101|   500.0|       4500.0|    ELETRONICOS|  SISTEMA_A|
|       2|2024-01-01|   Mouse Sem Fio|  50.0| Acessorios|       102|     5.0|         45.0|     ACESSORIOS|  SISTEMA_A|
|       3|2024-01-02|Teclado Mecanico| 200.0| Acessorios|       101|    20.0|        180.0|     ACESSORIOS|  SISTEMA_A|
|       4|2024-01-02|      Monitor 24| 900.0|Eletronicos|       103|    90.0|        810.0|    ELETRONICOS|  SISTEMA_A|
|       5|2024-01-03|   Cadeira Gamer|1200.0|     Moveis|       101|   120.0|       1080.0|         MOVEIS|  SISTEMA_A|
|       6|2024-01-03|        Mousepad|  

In [16]:
# condicional 

df_classificado = df_calculado.withColumn(
    "tipo_venda",
    when(col("valor") >= 3000, "Premium")
    .when(col("valor") > 1000, "Padrao")
    .otherwise("Basico")
)

In [17]:
df_classificado.select("produto", "valor", "tipo_venda").show()

+----------------+------+----------+
|         produto| valor|tipo_venda|
+----------------+------+----------+
|  Notebook Gamer|5000.0|   Premium|
|   Mouse Sem Fio|  50.0|    Basico|
|Teclado Mecanico| 200.0|    Basico|
|      Monitor 24| 900.0|    Basico|
|   Cadeira Gamer|1200.0|    Padrao|
|        Mousepad|  30.0|    Basico|
| Notebook Office|3000.0|   Premium|
+----------------+------+----------+



In [4]:
# Left Join
df_join = df_classificado.join(
    df_clientes,
    on="id_cliente", # Se a coluna tiver mesmo nome
    how="left"
)
df_relatorio_final = df_join.select(
    "id_venda", "data", "produto", "valor", "nome", "tipo_venda"
)
df_relatorio_final.show()

IndentationError: unexpected indent (2778071297.py, line 3)

In [21]:
# Left Join
df_join = df_classificado.join(
    df_clientes,
    on="id_cliente",
    how="right"
)
# Selecionar só o que importa para ficar limpo
df_relatorio_final = df_join.select(
    "id_venda", "produto", "valor", "nome"
)
df_relatorio_final.show()

+--------+----------------+------+--------------+
|id_venda|         produto| valor|          nome|
+--------+----------------+------+--------------+
|       5|   Cadeira Gamer|1200.0| Julia Pigosso|
|       3|Teclado Mecanico| 200.0| Julia Pigosso|
|       1|  Notebook Gamer|5000.0| Julia Pigosso|
|       7| Notebook Office|3000.0|Alisson Felipe|
|       2|   Mouse Sem Fio|  50.0|Alisson Felipe|
|       4|      Monitor 24| 900.0|   Chico Bento|
|    NULL|            NULL|  NULL|     Ana Julia|
+--------+----------------+------+--------------+



In [ ]:
logica_soma = sum("valor").over(Window.partitionBy("categoria"))

janela_ranking = Window.orderBy(logica_soma.desc())

df_direto = df_classificado.withColumn(
    "ranking_categoria",
    dense_rank().over(janela_ranking)
)

df_direto.select("categoria", "produto", "valor", "ranking_categoria").show()

+-----------+----------------+------+-----------------+
|  categoria|         produto| valor|ranking_categoria|
+-----------+----------------+------+-----------------+
|Eletronicos|  Notebook Gamer|5000.0|                1|
|Eletronicos|      Monitor 24| 900.0|                1|
|Eletronicos| Notebook Office|3000.0|                1|
|     Moveis|   Cadeira Gamer|1200.0|                2|
| Acessorios|   Mouse Sem Fio|  50.0|                3|
| Acessorios|Teclado Mecanico| 200.0|                3|
| Acessorios|        Mousepad|  30.0|                3|
+-----------+----------------+------+-----------------+



In [ ]:
janela_soma = Window.partitionBy("categoria")

df_com_total = df_classificado.withColumn(
    "total_da_categoria", 
    sum("valor").over(janela_soma)
)

janela_ranking = Window.orderBy(col("total_da_categoria").desc())

df_final = df_com_total.withColumn(
    "ranking_categoria", 
    dense_rank().over(janela_ranking)
)


df_final.select("categoria", "produto", "valor", "total_da_categoria", "ranking_categoria").show()

+-----------+----------------+------+------------------+-----------------+
|  categoria|         produto| valor|total_da_categoria|ranking_categoria|
+-----------+----------------+------+------------------+-----------------+
|Eletronicos|  Notebook Gamer|5000.0|            8900.0|                1|
|Eletronicos|      Monitor 24| 900.0|            8900.0|                1|
|Eletronicos| Notebook Office|3000.0|            8900.0|                1|
|     Moveis|   Cadeira Gamer|1200.0|            1200.0|                2|
| Acessorios|   Mouse Sem Fio|  50.0|             280.0|                3|
| Acessorios|Teclado Mecanico| 200.0|             280.0|                3|
| Acessorios|        Mousepad|  30.0|             280.0|                3|
+-----------+----------------+------+------------------+-----------------+



In [ ]:
df_rank.filter(col("ranking") == 1)\
.select("categoria", "produto", "valor").show()

+-----------+----------------+------+
|  categoria|         produto| valor|
+-----------+----------------+------+
| Acessorios|Teclado Mecanico| 200.0|
|Eletronicos|  Notebook Gamer|5000.0|
|     Moveis|   Cadeira Gamer|1200.0|
+-----------+----------------+------+



In [ ]:
# agrupamento e agregação

df_resumo = df_classificado.groupBy("categoria").agg(
    sum("valor").alias("total_faturado"),
    round(avg("valor"), 2).alias("ticket_medio"),
    count("id_venda").alias("qtd_vendas"),
    max("produto").alias("exemplo_produto_caro")
)

df_resumo.orderBy(col("total_faturado").desc()).show()

+-----------+--------------+------------+----------+--------------------+
|  categoria|total_faturado|ticket_medio|qtd_vendas|exemplo_produto_caro|
+-----------+--------------+------------+----------+--------------------+
|Eletronicos|        8900.0|     2966.67|         3|     Notebook Office|
|     Moveis|        1200.0|      1200.0|         1|       Cadeira Gamer|
| Acessorios|         280.0|       93.33|         3|    Teclado Mecanico|
+-----------+--------------+------------+----------+--------------------+

